In [1]:
import pandas as pd
import numpy as np
import csv as csv
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split
import time
import time
from datetime import date
import datetime

%matplotlib inline

In [2]:
train_df = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset.csv', header=0)  
train_df.head()

,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,0,0,0,13,0,0,13,0,False
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,3,16,0,83,0,43,83,4,False
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,0,0,0,16,0,3,16,0,False
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,0,16,0,85,0,9,85,0,False
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,1,17,0,43,0,0,43,0,False


Create columns with zeros
--------

In [3]:
dataframetest = pd.DataFrame({'mail_category':['mail_category_1', 'mail_category_10', 'mail_category_11',
       'mail_category_12']})
dataframetest.head()

,mail_category
0,mail_category_1
1,mail_category_10
2,mail_category_11
3,mail_category_12


In [4]:
all_mail_categories = ['mail_category_1', 'mail_category_10', 'mail_category_11',
       'mail_category_12', 'mail_category_13', 'mail_category_14',
       'mail_category_15', 'mail_category_16', 'mail_category_17',
       'mail_category_18', 'mail_category_2', 'mail_category_3',
       'mail_category_4', 'mail_category_5', 'mail_category_6',
       'mail_category_7', 'mail_category_8', 'mail_category_9']


dataFrameMailCategoryDiff = np.setdiff1d(all_mail_categories,np.unique(dataframetest.mail_category)) 
print 'Difference columns not found on dataframe: ',dataFrameMailCategoryDiff
dummiesMail_category =  pd.get_dummies(dataframetest.mail_category,prefix='col')
dataframetest = pd.concat([dataframetest, dummiesMail_category], axis=1)
dataframetest = dataframetest.drop(['mail_category'],axis=1) #drop the mail_category
dataframetest.head()

Difference columns not found on dataframe:  ['mail_category_13' 'mail_category_14' 'mail_category_15'
 'mail_category_16' 'mail_category_17' 'mail_category_18' 'mail_category_2'
 'mail_category_3' 'mail_category_4' 'mail_category_5' 'mail_category_6'
 'mail_category_7' 'mail_category_8' 'mail_category_9']


,col_mail_category_1,col_mail_category_10,col_mail_category_11,col_mail_category_12
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0


#### Now we concatenate the other dummies not found on the dataframe

In [5]:
for mailCategoryItem in map(lambda x:"col_"+x,dataFrameMailCategoryDiff):
    dataframetest = pd.concat([dataframetest,pd.DataFrame({mailCategoryItem: np.zeros(dataframetest.shape[0])})],axis=1)
dataframetest.head()

,col_mail_category_1,col_mail_category_10,col_mail_category_11,col_mail_category_12,col_mail_category_13,col_mail_category_14,col_mail_category_15,col_mail_category_16,col_mail_category_17,col_mail_category_18,col_mail_category_2,col_mail_category_3,col_mail_category_4,col_mail_category_5,col_mail_category_6,col_mail_category_7,col_mail_category_8,col_mail_category_9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
